In [32]:
# -*- coding: utf-8 -*-

from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import re
import requests
from datetime import datetime, date, timedelta
import traceback
import threading
import time
from collections import deque
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pyperclip

# ==============================================================================
# 1. FUNÇÕES AUXILIARES
# ==============================================================================
def formatar_brl(valor):
    """Formata um valor numérico para o padrão monetário brasileiro (R$ X.XXX,XX)."""
    try:
        s = f"{float(valor):,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
        return s
    except (ValueError, TypeError):
        return "0,00"

def formatar_me(valor):
    """Formata um valor numérico para o padrão de moeda estrangeira (X,XXX.XX)."""
    try:
        return f"{float(parse_input(valor)):,.2f}"
    except (ValueError, TypeError):
        return "0.00"

def formatar_taxa_precisa(valor):
    """Formata uma taxa com alta precisão e padrão brasileiro (X.XXX,XXXXXXXXXXXXXXXX)."""
    try:
        s = f"{float(valor):,.16f}".replace(',', 'X').replace('.', ',').replace('X', '.')
        return s
    except (ValueError, TypeError):
        return "0,0000000000000000"

def formatar_generico(valor, casas_decimais=4):
    """Formata um número para o padrão brasileiro com um número customizado de casas decimais."""
    try:
        return f"{float(valor):,.{casas_decimais}f}".replace(',', 'X').replace('.', ',').replace('X', '.')
    except (ValueError, TypeError):
        return "0,00"


def parse_input(valor):
    """Converte um input (string, int, float) para float, limpando caracteres não numéricos."""
    try:
        if isinstance(valor, (int, float)):
            return float(valor)
        s = str(valor).replace('.', '').replace(',', '.')
        s = re.sub(r'[^\d.]', '', s)
        return float(s)
    except (ValueError, TypeError):
        return 0.0

# Objeto global para exibir logs de erros na interface
debug_output = widgets.Output(layout={'border': '1px solid red', 'padding': '5px'})

# ==============================================================================
# 2. CLASSES DOS MÓDULOS
# ==============================================================================
class ModuloBase:
    """Classe base para todos os módulos da interface."""
    def __init__(self):
        self.layout = widgets.VBox()

    def get_layout(self):
        return self.layout

class ModuloCompra(ModuloBase):
    def __init__(self, app_ref):
        super().__init__()
        self.app_ref = app_ref
        self.taxa_comercial_hidden = widgets.FloatText()
        self.info_operacao = widgets.HTML("<i>Preencha 2 dos 3 campos <b>em negrito</b> abaixo e clique em CALCULAR.</i>")
        self.valor_me_local = widgets.FloatText(style={'description_width': 'initial'})
        self.valor_reais_local = widgets.FloatText(style={'description_width': 'initial'})
        self.taxa_cambial_local = widgets.FloatText(style={'description_width': 'initial'})
        label_layout = widgets.Layout(width='160px', display='flex', align_items='center')
        box_valor_me = widgets.HBox([widgets.HTML("<b>Valor em M.E:</b>", layout=label_layout), self.valor_me_local])
        box_valor_reais = widgets.HBox([widgets.HTML("<b>Valor em R$:</b>", layout=label_layout), self.valor_reais_local])
        box_taxa_cambial = widgets.HBox([widgets.HTML("<b>Taxa Cambial (R$):</b>", layout=label_layout), self.taxa_cambial_local])
        campos_calculo_box = widgets.VBox([box_valor_me, box_valor_reais, box_taxa_cambial])
        self.iof_compra = widgets.Dropdown(options=[('ISENTO', 0.0), ('0,38%', 0.0038)], value=0.0038, description='IOF:')
        self.tarifa_compra = widgets.Dropdown(options=[('ISENTO', 0.0), ('R$ 250,00', 250.0)], value=250.0, description='Tarifa:')
        self.calcular_button = widgets.Button(description='CALCULAR', button_style='primary', icon='calculator', tooltip="Calcular com os valores informados")
        self.limpar_button = widgets.Button(description='LIMPAR', button_style='danger', icon='trash', tooltip="Limpar os campos deste módulo")
        self.output_display = widgets.HTML()
        self.calcular_button.on_click(self.calcular)
        self.limpar_button.on_click(self.on_limpar_click)
        botoes_box = widgets.HBox([self.calcular_button, self.limpar_button])
        self.layout.children = [self.info_operacao, campos_calculo_box, self.iof_compra, self.tarifa_compra, botoes_box, widgets.HTML("<hr>"), self.output_display]
        self.limpar_state()

    def calcular(self, b):
        self.app_ref.increment_click_count()
        with debug_output:
            clear_output(wait=True)
            try:
                valor_me = self.valor_me_local.value or 0
                valor_rs = self.valor_reais_local.value or 0
                taxa = self.taxa_cambial_local.value or 0
                taxa_comercial_ref = self.taxa_comercial_hidden.value or 0
                if sum([valor_me > 0, valor_rs > 0, taxa > 0]) != 2:
                    self.output_display.value = "<b style='color:red;'>Erro: Por favor, preencha exatamente 2 dos 3 campos.</b>"
                    return
                if valor_me > 0 and taxa > 0: valor_rs = valor_me * taxa
                elif valor_rs > 0 and taxa > 0: valor_me = valor_rs / taxa if taxa != 0 else 0
                elif valor_me > 0 and valor_rs > 0: taxa = round(valor_rs / valor_me if valor_me != 0 else 0, 4); self.taxa_cambial_local.value = taxa
                valor_bruto_reais = valor_rs; moeda = self.app_ref.moeda_selecionada.value; iof_pct, tarifa = self.iof_compra.value, self.tarifa_compra.value
                vet_calculado = taxa * (1 - iof_pct); valor_iof = valor_bruto_reais * iof_pct; total_creditado = valor_bruto_reais - valor_iof - tarifa
                receita_html = ""
                if taxa_comercial_ref > 0:
                    receita_estimada = (taxa_comercial_ref - taxa) * valor_me
                    self.app_ref.add_revenue(receita_estimada)
                    warning_message = " <b style='color:red;'>OPERAÇAO COM PREJUÍZO</b>" if taxa >= taxa_comercial_ref else ""
                    receita_html = f"<b>RECEITA ESTIMADA R$:</b> <span style='color:blue; font-weight:bold;'>{formatar_brl(receita_estimada)}</span>{warning_message}<br>"
                self.output_display.value = f"""<h4>RESUMO OPERAÇÃO COMPRA</h4>
                                                <b>Valor em Moeda Estrangeira ({moeda}):</b> <span style='color:black; font-weight:bold;'>{formatar_me(valor_me)}</span><br>
                                                <b>Taxa Cambial:</b> R$ {formatar_generico(taxa, 4)}<br>
                                                <b>Valor Bruto em Reais (R$):</b> R$ {formatar_brl(valor_bruto_reais)}<br>
                                                {receita_html}
                                                <b>V.E.T. (Valor Efetivo Total): <span style='color:purple;font-weight:bold;'>R$ {formatar_generico(vet_calculado, 4)}</span></b><hr>
                                                <b>IOF ({iof_pct*100:.2f}%):</b> {'ISENTO' if valor_iof == 0 else f"<span style='color:red;'>- R$ {formatar_brl(valor_iof)}</span>"}<br>
                                                <b>Tarifa:</b> {'ISENTO' if tarifa == 0 else f"<span style='color:red;'>- R$ {formatar_brl(tarifa)}</span>"}<br><hr>
                                                <b>Valor Creditado:</b> <span style='color:green; font-weight:bold;'>R$ {formatar_brl(total_creditado)}</span>"""
            except Exception as e: print(f"Erro em ModuloCompra.calcular:\n{traceback.format_exc()}")
    def on_limpar_click(self, b): self.limpar_state()
    def limpar_state(self): self.info_operacao.value = "<i>Preencha 2 dos 3 campos <b>em negrito</b> abaixo e clique em CALCULAR.</i>"; self.valor_me_local.value = 0; self.valor_reais_local.value = 0; self.taxa_cambial_local.value = 0; self.output_display.value = ""; self.taxa_comercial_hidden.value = 0

class ModuloVenda(ModuloBase):
    def __init__(self, app_ref):
        super().__init__()
        self.app_ref = app_ref
        self.info_operacao = widgets.HTML("<i>Preencha 2 dos 3 campos <b>em negrito</b> abaixo e clique em CALCULAR.</i>")
        self.valor_me_local = widgets.FloatText(style={'description_width': 'initial'}); self.valor_reais_local = widgets.FloatText(style={'description_width': 'initial'}); self.taxa_cambial_local = widgets.FloatText(style={'description_width': 'initial'}); self.taxa_comercial_hidden = widgets.FloatText()
        label_layout = widgets.Layout(width='160px', display='flex', align_items='center')
        campos_calculo_box = widgets.VBox([widgets.HBox([widgets.HTML("<b>Valor em M.E:</b>", layout=label_layout), self.valor_me_local]), widgets.HBox([widgets.HTML("<b>Valor em R$:</b>", layout=label_layout), self.valor_reais_local]), widgets.HBox([widgets.HTML("<b>Taxa Cambial (R$):</b>", layout=label_layout), self.taxa_cambial_local])])
        self.iof_venda = widgets.Dropdown(options=[('ISENTO', 0.0), ('0,38%', 0.0038), ('1,10%', 0.011), ('3,5%', 0.035)], value=0.011, description='IOF:')
        self.ir_venda = widgets.Dropdown(options=[('ISENTO',0.0), ('10% (CREDOR)', 0.10), ('15% (CREDOR)',0.15), ('25% (CREDOR)', 0.25), ('11,11111% (DEVEDOR)',0.1111111), ('17,64706% (DEVEDOR)',0.1764706), ('33,33330% (DEVEDOR)', 0.333333)], value=0.0, description='I.R:')
        self.tarifa_venda = widgets.Dropdown(options=[('ISENTO',0.0), ('R$ 250,00',250.0)], value=250.0, description='Tarifa:')
        self.ir_warning_label = widgets.HTML(value="", layout={'visibility': 'hidden', 'padding': '0px 10px'})
        self.calcular_button = widgets.Button(description='CALCULAR', button_style='primary', icon='calculator', tooltip="Calcular com os valores informados")
        self.limpar_button = widgets.Button(description='LIMPAR', button_style='danger', icon='trash', tooltip="Limpar os campos deste módulo")
        self.output_display = widgets.HTML()
        self.calcular_button.on_click(self.calcular); self.limpar_button.on_click(self.on_limpar_click); self.ir_venda.observe(self._on_ir_change, names='value')
        botoes_box = widgets.HBox([self.calcular_button, self.limpar_button])
        self.layout.children = [self.info_operacao, campos_calculo_box, self.iof_venda, widgets.HBox([self.ir_venda, self.ir_warning_label]), self.tarifa_venda, botoes_box, widgets.HTML("<hr>"), self.output_display]
        self.limpar_state()

    def _on_ir_change(self, change):
        new_val = change['new']
        devedor_types = [0.1111111, 0.1764706, 0.333333]

        # Aviso genérico que a Ptax pode ser usada na base de cálculo do I.R.
        ptax_warning = "<span style='color:red;font-size:11px;'>↳ Cálculo do I.R. considera a Maior Ptax (D-2), se aplicável.</span>"

        # Aviso específico sobre o tipo de débito do IR Devedor
        devedor_warning = "<b style='color:red;font-size:11px;'>Atenção! Com esta alíquota, o I.R. será debitado à parte.</b>"

        if new_val == 0.0:
            # Se for ISENTO, esconde o aviso.
            self.ir_warning_label.layout.visibility = 'hidden'
            self.ir_warning_label.value = ""
        elif new_val in devedor_types:
            # Se for DEVEDOR, mostra ambos os avisos.
            self.ir_warning_label.value = f"{devedor_warning}<br>{ptax_warning}"
            self.ir_warning_label.layout.visibility = 'visible'
        else:
            # Se for CREDOR (ou qualquer outro não isento), mostra apenas o aviso da Ptax.
            self.ir_warning_label.value = ptax_warning
            self.ir_warning_label.layout.visibility = 'visible'

    def calcular(self, b):
        self.app_ref.increment_click_count()
        with debug_output:
            clear_output(wait=True)
            try:
                valor_me = self.valor_me_local.value or 0
                valor_rs = self.valor_reais_local.value or 0
                taxa = self.taxa_cambial_local.value or 0
                taxa_comercial_ref = self.taxa_comercial_hidden.value or 0

                if sum([valor_me > 0, valor_rs > 0, taxa > 0]) != 2: self.output_display.value = "<b style='color:red;'>Erro: Preencha 2 de 3 campos.</b>"; return
                if valor_me > 0 and taxa > 0: valor_rs = valor_me * taxa
                elif valor_rs > 0 and taxa > 0: valor_me = valor_rs / taxa if taxa != 0 else 0
                elif valor_me > 0 and valor_rs > 0: taxa = round(valor_rs / valor_me if valor_me != 0 else 0, 4); self.taxa_cambial_local.value = taxa

                valor_bruto_reais = valor_rs
                moeda = self.app_ref.moeda_selecionada.value
                iof_pct, ir_pct, tarifa = self.iof_venda.value, self.ir_venda.value, self.tarifa_venda.value
                vet_calculado = taxa * (1 + iof_pct)

                # --- LÓGICA DE VERIFICAÇÃO PTAX D-2 ---
                base_de_calculo_ir = taxa
                ptax_d2_html = ""
                ptax_check_result = self.app_ref._get_venda_ir_base_rate(moeda, taxa)

                if ptax_check_result:
                    base_de_calculo_ir = ptax_check_result["rate"]
                    formatted_rate = formatar_generico(ptax_check_result["rate"], 4)
                    ptax_d2_html = (
                        f"<div style='margin-top: 5px; margin-bottom: 5px;'>"
                        f"<span style='background-color: yellow; color: red; font-weight: bold; padding: 4px 6px; border-radius: 4px;'>"
                        f"MAIOR ENTRE PTAX (D-2) R$ {formatted_rate}"
                        f"</span>"
                        f"</div>"
                    )
                # --- FIM DA LÓGICA ---

                valor_base_ir = valor_me * base_de_calculo_ir
                valor_ir = valor_base_ir * ir_pct

                valor_base_iof = valor_bruto_reais - valor_ir if ir_pct in [0.10, 0.15, 0.25] else valor_bruto_reais
                valor_iof = valor_base_iof * iof_pct

                total_operacao = valor_bruto_reais + valor_iof + valor_ir + tarifa
                receita_html = ""
                if taxa_comercial_ref > 0:
                    receita_estimada = (taxa - taxa_comercial_ref) * valor_me
                    self.app_ref.add_revenue(receita_estimada)
                    warning_message = " <b style='color:red;'>OPERAÇAO COM PREJUÍZO</b>" if taxa <= taxa_comercial_ref else ""
                    receita_html = f"<b>RECEITA ESTIMADA R$:</b> <span style='color:blue;font-weight:bold;'>{formatar_brl(receita_estimada)}</span>{warning_message}<br>"

                self.output_display.value = (f"<h4>RESUMO OPERAÇÃO VENDA</h4>"
                                              f"<b>Valor em Moeda Estrangeira ({moeda}):</b> <span style='color:black; font-weight:bold;'>{formatar_me(valor_me)}</span><br>"
                                              f"<b>Taxa Cambial:</b> R$ {formatar_generico(taxa, 4)}<br>"
                                              f"<b>Valor Base em Reais (R$):</b> R$ {formatar_brl(valor_bruto_reais)}<br>"
                                              f"{receita_html}"
                                              f"<b>V.E.T. (Valor Efetivo Total): <span style='color:purple;font-weight:bold;'>R$ {formatar_generico(vet_calculado, 4)}</span></b><hr>"
                                              f"<b>IOF ({iof_pct*100:.2f}%):</b> {'ISENTO' if valor_iof==0 else f'+ R$ {formatar_brl(valor_iof)}'}<br>"
                                              f"<b>I.R.:</b> {'ISENTO' if valor_ir==0 else f'+ R$ {formatar_brl(valor_ir)}'}<br>"
                                              f"{ptax_d2_html}"
                                              f"<b>Tarifa:</b> {'ISENTO' if tarifa==0 else f'+ R$ {formatar_brl(tarifa)}'}<br><hr>"
                                              f"<b>Custo Total:</b> <span style='color:red;font-weight:bold;'>R$ {formatar_brl(total_operacao)}</span>")
            except Exception: print(f"Erro em ModuloVenda.calcular:\n{traceback.format_exc()}")

    def on_limpar_click(self, b): self.limpar_state()

    def limpar_state(self):
        self.valor_me_local.value = 0
        self.valor_reais_local.value = 0
        self.taxa_cambial_local.value = 0
        self.output_display.value = ""
        self.taxa_comercial_hidden.value = 0

class ModuloValorEmReais(ModuloBase):
    def __init__(self, app_ref):
        super().__init__()
        self.app_ref=app_ref
        self.ultimo_valor_boleto=0.0
        self.valor_brl=widgets.Text(description='Valor em R$:',placeholder='Ex: 10000.00')
        self.iof_dropdown=widgets.Dropdown(options=[('ISENTO',0.0), ('0,38%', 0.0038), ('1,10%',0.011),('3,5%',0.035)],value=0.0,description='IOF:')
        self.ir_dropdown=widgets.Dropdown(options=[('ISENTO',0.0),('10% (CREDOR)', 0.10), ('15% (CREDOR)',0.15), ('25% (CREDOR)', 0.25), ('11,11111% (DEVEDOR)',0.1111111), ('17,64706% (DEVEDOR)',0.1764706), ('33,33330% (DEVEDOR)', 0.333333)],value=0.0,description='I.R:')
        self.tarifa_dropdown=widgets.Dropdown(options=[('ISENTO',0.0),('R$ 250,00',250.0)],value=0.0,description='Tarifa:')
        self.ir_warning_label=widgets.HTML(value="",layout={'visibility':'hidden','padding':'0px 10px'})
        self.calcular_button=widgets.Button(description='CALCULAR',button_style='success',icon='calculator')
        self.limpar_button=widgets.Button(description='LIMPAR',button_style='danger',icon='trash')
        self.output_display=widgets.HTML(value="<p><i>Insira o valor em R$ e clique em CALCULAR.</i></p>")
        self.abrir_taxa_button=widgets.Button(description='ABRIR TAXA',icon='plus-square',button_style='info')

        self.taxa_calc_valor_rs=widgets.FloatText(description='VALOR R$:')
        self.taxa_calc_valor_me=widgets.FloatText(description='VALOR M.E:')
        self.taxa_calc_calcular_button=widgets.Button(description='CALCULAR',button_style='primary',icon='calculator')
        self.taxa_calc_limpar_button=widgets.Button(description='LIMPAR',button_style='warning',icon='trash')
        self.taxa_calc_output=widgets.HTML()
        self.taxa_calculator_box=widgets.VBox([
            widgets.HTML("<hr><h4>Calculadora de Taxa Simples</h4>"),
            self.taxa_calc_valor_rs,
            self.taxa_calc_valor_me,
            widgets.HBox([self.taxa_calc_calcular_button,self.taxa_calc_limpar_button]),
            self.taxa_calc_output
        ], layout={'display':'none','border':'1px solid #ccc','padding':'10px','margin-top':'10px'})

        self.calcular_button.on_click(self.calcular_custos)
        self.limpar_button.on_click(self.limpar)
        self.ir_dropdown.observe(self._on_ir_change,names='value')
        self.abrir_taxa_button.on_click(self._toggle_taxa_calculator)
        self.taxa_calc_calcular_button.on_click(self._calculate_taxa_simples)
        self.taxa_calc_limpar_button.on_click(self._limpar_taxa_calculator)

        self.layout.children=[
            widgets.HTML("<p>Precisa garantir que a operação não custe nenhum centavo a mais do que o cliente tem em conta ? Sem problema, calcule abaixo.</p>"),
            self.valor_brl,
            self.iof_dropdown,
            widgets.HBox([self.ir_dropdown,self.ir_warning_label]),
            self.tarifa_dropdown,
            widgets.HBox([self.calcular_button,self.limpar_button,self.abrir_taxa_button]),
            self.output_display,
            self.taxa_calculator_box
        ]

    def _toggle_taxa_calculator(self, b):
        if self.taxa_calculator_box.layout.display=='none':
            valor_arredondado = round(self.ultimo_valor_boleto, 2)
            self.taxa_calc_valor_rs.value = valor_arredondado if valor_arredondado > 0 else 0
            self.taxa_calculator_box.layout.display='block'
            self.abrir_taxa_button.description='VOLTAR'
            self.abrir_taxa_button.icon='undo'
        else:
            self.taxa_calculator_box.layout.display='none'
            self.abrir_taxa_button.description='ABRIR TAXA'
            self.abrir_taxa_button.icon='plus-square'

    def _calculate_taxa_simples(self, b):
        self.app_ref.increment_click_count()
        valor_rs = self.taxa_calc_valor_rs.value or 0
        valor_me = self.taxa_calc_valor_me.value or 0
        if valor_me == 0:
            self.taxa_calc_output.value="<b style='color:red;'>Valor M.E não pode ser zero.</b>"
            return
        resultado_formatado = formatar_taxa_precisa(valor_rs / valor_me)
        self.taxa_calc_output.value = (f"<b>Resultado da Taxa R$:</b> "
                                      f"<span style='font-size:1.2em;color:green;font-weight:bold;'>{resultado_formatado}</span>")

    def _limpar_taxa_calculator(self, b):
        self.taxa_calc_valor_rs.value = 0
        self.taxa_calc_valor_me.value = 0
        self.taxa_calc_output.value = ""

    def _on_ir_change(self, change):
        # Este módulo não utiliza PTAX D-2, portanto nenhum aviso é adicionado aqui.
        self.ir_warning_label.layout.visibility = 'hidden'

    def calcular_custos(self, b):
        self.app_ref.increment_click_count()
        with debug_output:
            clear_output(wait=True)
            self.output_display.value = ""
            try:
                # 1. Obter valores dos widgets de entrada
                valor_na_conta = parse_input(self.valor_brl.value)
                if not valor_na_conta or valor_na_conta <= 0:
                    self.output_display.value = "<b style='color:red'>O Valor em R$ deve ser preenchido e maior que zero.</b>"
                    return

                iof_pct = self.iof_dropdown.value
                ir_pct = self.ir_dropdown.value
                tarifa = self.tarifa_dropdown.value

                credor_ir_types = [0.10, 0.15, 0.25]
                devedor_ir_types = [0.1111111, 0.1764706, 0.333333]

                # 2. Calcular o 'Valor do Boleto' conforme a especificação.

                # A base para o cálculo começa com o valor em conta menos a tarifa.
                valor_base_calculo = valor_na_conta - tarifa

                # Para IR do tipo CREDOR, o valor do IR é subtraído da base antes da divisão pelo IOF.
                # O IR CREDOR é calculado sobre o valor total na conta.
                if ir_pct in credor_ir_types:
                    ir_credor_valor = valor_na_conta * ir_pct
                    valor_base_calculo -= ir_credor_valor

                # O divisor de IOF é sempre 1 + a alíquota de IOF.
                # Para IR do tipo DEVEDOR, a alíquota de IR também entra no divisor (cálculo "gross up").
                divisor = 1 + iof_pct
                if ir_pct in devedor_ir_types:
                    divisor += ir_pct

                valor_boleto = valor_base_calculo / divisor if divisor != 0 else 0
                self.ultimo_valor_boleto = valor_boleto

                # 3. Calcular os valores dos custos para exibição nos detalhes.
                valor_iof = valor_boleto * iof_pct

                # O valor do IR depende do tipo.
                if ir_pct in credor_ir_types:
                    # Para CREDOR, o valor já foi calculado sobre o total da conta.
                    valor_ir = valor_na_conta * ir_pct
                elif ir_pct in devedor_ir_types:
                    # Para DEVEDOR, é calculado sobre o valor do boleto.
                    valor_ir = valor_boleto * ir_pct
                else: # ISENTO
                    valor_ir = 0

                # O 'Valor Base' para exibição é o valor após a tarifa e, se aplicável, o IR inicial (CREDOR).
                valor_base_display = valor_na_conta - tarifa
                if ir_pct in credor_ir_types:
                    valor_base_display -= valor_ir

                # O 'Custo Total' deve ser a soma de todas as partes, que deve resultar no valor original em conta.
                custo_total = valor_boleto + valor_iof + valor_ir + tarifa

                # 4. Montar o HTML de saída com os resultados formatados.
                self.output_display.value = (
                    f"<h4>✅ Resultado</h4>"
                    f"<b>Valor na Conta:</b>&nbsp;R$ {formatar_brl(valor_na_conta)}<br>"
                    f"<b>Valor do Boleto:</b>&nbsp;<span style='color:blue;font-weight:bold;'>R$ {formatar_brl(valor_boleto)}</span><br><hr>"
                    f"<b>Detalhes (R$):</b><br>"
                    f"Valor Base (após tarifa e IR inicial): R$ {formatar_brl(valor_base_display)}<br>"
                    f"IOF ({iof_pct*100:.2f}%): {'ISENTO' if valor_iof == 0 else f'+ R$ {formatar_brl(valor_iof)}'}<br>"
                    f"I.R.: {'ISENTO' if valor_ir == 0 else f'+ R$ {formatar_brl(valor_ir)}'}<br>"
                    f"Tarifa: {'ISENTO' if tarifa == 0 else f'+ R$ {formatar_brl(tarifa)}'}<br><hr>"
                    f"<b>Custo Total:</b>&nbsp;<span style='color:red;font-weight:bold;'>R$ {formatar_brl(custo_total)}</span>"
                )

            except Exception as e:
                print(f"Erro em ModuloValorEmReais.calcular_custos:\n{traceback.format_exc()}")

    def limpar(self, b): self.valor_brl.value=""; self.iof_dropdown.value=0.0; self.ir_dropdown.value=0.0; self.tarifa_dropdown.value=0.0; self.ultimo_valor_boleto=0.0; self.output_display.value="<p><i>Campos limpos.</i></p>"; self._limpar_taxa_calculator(None); self.taxa_calculator_box.layout.display='none'; self.abrir_taxa_button.description='ABRIR TAXA'; self.abrir_taxa_button.icon='plus-square'

class ModuloConcomitante(ModuloBase):
    def __init__(self, app_ref):
        super().__init__()
        self.app_ref = app_ref
        self.info_visible = False
        self.ponta_venda_valor = widgets.Text(description='Valor Incidente:', placeholder='Ex: 10000,00')
        self.ponta_venda_ir = widgets.Dropdown(options=[('ISENTO',0.0),('10% (CREDOR)', 0.10), ('15% (CREDOR)',0.15), ('25% (CREDOR)', 0.25), ('11,11111% (DEVEDOR)',0.1111111), ('17,64706% (DEVEDOR)',0.1764706), ('33,33330% (DEVEDOR)', 0.333333)],description='I.R:')
        self.ir_warning_label = widgets.HTML(value="", layout={'visibility': 'hidden', 'padding': '0px 10px'})
        self.ponta_venda_iof = widgets.Dropdown(options=[('ISENTO',0.0), ('0,38%', 0.0038), ('1,10%',0.011),('3,5%',0.035)],description='IOF Venda:')
        self.ponta_venda_tarifa = widgets.Dropdown(options=[('ISENTO',0.0),('R$ 250,00',250.0)],description='Tarifa Venda:')
        self.ponta_compra_iof = widgets.Dropdown(options=[('0,38%',0.0038)],description='IOF Compra:')
        self.ponta_compra_tarifa = widgets.Dropdown(options=[('ISENTO',0.0),('R$ 250,00',250.0)],description='Tarifa Compra:')
        self.calcular_button = widgets.Button(description='CALCULAR', button_style='primary', icon='calculator')
        self.limpar_button = widgets.Button(description='LIMPAR', button_style='danger', icon='trash')
        self.outputs = {k: widgets.HTML() for k in ["operacao", "venda", "compra", "final"]}
        self.info_button = widgets.Button(
            icon='info-circle',
            tooltip='O que é uma operação concomitante?',
            layout=widgets.Layout(width='auto', border='none', background='transparent')
        )
        self.info_output = widgets.Output()
        self.info_button.on_click(self._toggle_info_message)
        self.ponta_venda_ir.observe(self._on_ir_change, names='value')
        self.calcular_button.on_click(self.calcular)
        self.limpar_button.on_click(self.limpar)
        ponta_venda_title = widgets.HBox([widgets.HTML("<b>PONTA VENDA:</b>"), self.info_button])
        self.layout.children = [
            ponta_venda_title,
            self.info_output,
            self.ponta_venda_valor,
            widgets.HBox([self.ponta_venda_ir, self.ir_warning_label]),
            self.ponta_venda_iof, self.ponta_venda_tarifa,
            widgets.HTML("<br><b>PONTA COMPRA:</b>"), self.ponta_compra_iof, self.ponta_compra_tarifa,
            widgets.HBox([self.calcular_button, self.limpar_button]),
            widgets.HTML("<hr>"), self.outputs["operacao"], widgets.HTML("<hr>"),
            self.outputs["venda"], widgets.HTML("<hr>"), self.outputs["compra"],
            widgets.HTML("<hr>"), self.outputs["final"]
        ]
        self.limpar(None)

    def _toggle_info_message(self, b):
        self.info_visible = not self.info_visible
        with self.info_output:
            clear_output()
            if self.info_visible:
                message_html = """
                <div style='border: 1px solid #007bff; padding: 10px; border-radius: 5px; background-color: #f0f7ff; margin-top: 5px; margin-bottom: 5px;'>
                    <b>Operação Concomitante:</b>
                    <p style='margin: 5px 0 0 0;'>Refere-se a um tipo de contrato que não envolve a movimentação real de dinheiro entre duas partes,
                    mas sim uma formalização via câmbio sem a troca efetiva de recursos. Essa operação também é conhecida
                    como câmbio simultâneo ou câmbio simbólico.</p>
                </div>
                """
                display(HTML(message_html))

    def _on_ir_change(self, c):
        if c['new'] in [0.1111111, 0.1764706, 0.333333]:
            self.ir_warning_label.value = "<b style='color:red;font-size:11px;'>Atenção! Ao selecionar esta alíquota, o valor de I.R será debitado à parte. Para debitar I.R do principal, escolha uma alíquota CREDOR.</b>"
            self.ir_warning_label.layout.visibility = 'visible'
        else:
            self.ir_warning_label.layout.visibility = 'hidden'

    def calcular(self, b):
        self.app_ref.increment_click_count()
        with debug_output:
            clear_output(wait=True)
            try:
                venda_valor = parse_input(self.ponta_venda_valor.value)
                venda_ir_pct = self.ponta_venda_ir.value
                venda_iof_pct = self.ponta_venda_iof.value
                tarifa_venda = self.ponta_venda_tarifa.value
                compra_iof_pct = self.ponta_compra_iof.value
                tarifa_compra = self.ponta_compra_tarifa.value

                valor_boleto_venda = venda_valor * (1 - venda_ir_pct) if venda_ir_pct in [0.10, 0.15, 0.25] else venda_valor

                if venda_iof_pct == 0.0:
                    valor_boleto_compra = 0.0
                else:
                    valor_base_gross_up = valor_boleto_venda * venda_iof_pct
                    valor_boleto_compra = valor_base_gross_up * (1 + 0.0038145)

                valor_iof_venda = valor_boleto_venda * venda_iof_pct
                valor_ir_venda = venda_valor * venda_ir_pct
                valor_iof_compra = valor_boleto_compra * compra_iof_pct

                valor_real_enviado = valor_boleto_venda - valor_iof_venda - tarifa_venda - valor_iof_compra - tarifa_compra

                total_debitado = valor_boleto_venda + valor_ir_venda + valor_iof_venda + tarifa_venda
                total_creditado = valor_boleto_compra - valor_iof_compra - tarifa_compra
                todos_custos = valor_ir_venda + valor_iof_venda + tarifa_venda + valor_iof_compra + tarifa_compra
                total_operacao = todos_custos + valor_boleto_venda

                ir_venda_label = [label for label, val in self.ponta_venda_ir.options if val == venda_ir_pct][0]
                iof_venda_label = [label for label, val in self.ponta_venda_iof.options if val == venda_iof_pct][0]
                ir_display = f"R$ {formatar_brl(valor_ir_venda)}" if valor_ir_venda != 0 else 'ISENTO'
                iof_display = f"R$ {formatar_brl(valor_iof_venda)}" if valor_iof_venda != 0 else 'ISENTO'
                tarifa_venda_display = f"R$ {formatar_brl(tarifa_venda)}" if tarifa_venda != 0 else 'ISENTO'

                self.outputs["operacao"].value = (
                    f"<h4>VALORES DA OPERAÇÃO</h4>"
                    f"<b>Boleto VENDA:</b> R$ {formatar_brl(valor_boleto_venda)}<br>"
                    f"<b>Boleto COMPRA:</b> R$ {formatar_brl(valor_boleto_compra)}"
                )
                self.outputs["venda"].value = (
                    f"<h4>RESUMO - PONTA VENDA</h4>"
                    f"<b>Boleto VENDA:</b> R$ {formatar_brl(valor_boleto_venda)}<br>"
                    f"<b>I.R. ({ir_venda_label}):</b> {ir_display}<br>"
                    f"<b>IOF ({iof_venda_label}):</b> {iof_display}<br>"
                    f"<b>Tarifa:</b> {tarifa_venda_display}<br>"
                    f"<b>Total DEBITADO:</b><span style='color:red;font-weight:bold;'>R$ {formatar_brl(total_debitado)}</span>"
                )
                self.outputs["compra"].value = (
                    f"<h4>RESUMO - PONTA COMPRA</h4>"
                    f"<b>IOF GROSS UP:</b> R$ {formatar_brl(valor_boleto_compra)}<br>"
                    f"<b>NET IOF COMPRA R$:</b> {formatar_brl(valor_boleto_compra - valor_iof_compra)}<br>"
                    f"<b>IOF Pta Compra (0,38%):</b><span style='color:red;'>- R$ {formatar_brl(valor_iof_compra)}</span><br>"
                    f"<b>Tarifa Compra:</b><span style='color:red;'>- R$ {formatar_brl(tarifa_compra)}</span><br>"
                    f"<b>Total CREDITADO:</b><span style='color:green;font-weight:bold;'>R$ {formatar_brl(total_creditado)}</span>"
                )
                self.outputs["final"].value = (
                    f"<h4>FINAL</h4>"
                    f"<b>VALOR INCIDENTE:</b> R$ {formatar_brl(venda_valor)}<br>"
                    f"<b>TOTAL DA OPERAÇÃO:</b><span style='color:red;font-weight:bold;'>R$ {formatar_brl(total_operacao)}</span><br>"
                    f"<b>TODOS OS CUSTOS:</b><span style='color:red;font-weight:bold;'>R$ {formatar_brl(todos_custos)}</span><br>"
                    f"<b>VALOR REAL ENVIADO:</b><span style='color:green;font-weight:bold;'>R$ {formatar_brl(valor_real_enviado)}</span>"
                )
            except Exception as e:
                for out in self.outputs.values(): out.value = ""
                print(f"Erro no Módulo Concomitante:\n{traceback.format_exc()}")

    def limpar(self, b):
        self.ponta_venda_valor.value = ""; self.ponta_venda_ir.value = 0.0; self.ponta_venda_iof.value = 0.0; self.ponta_venda_tarifa.value = 0.0; self.ponta_compra_tarifa.value = 0.0
        for out in self.outputs.values(): out.value = ""
        self.outputs["operacao"].value = "<p><i>Insira os valores.</i></p>"
        self.info_visible = False
        with self.info_output:
            clear_output()

class ModuloSpreadConcorrencia(ModuloBase):
    def __init__(self, app_ref):
        super().__init__()
        self.app_ref = app_ref
        style = {'description_width': 'initial'}

        info_html = widgets.HTML("<i>Quer saber o spread que a concorrência está operando? Insira a taxa abaixo e clique em compra ou venda, ou preencha os outros campos.</i>")
        self.tarifa_custo_input = widgets.FloatText(description='TARIFA/ CUSTO CONCORRENCIA: R$', style=style)
        self.valor_me_input = widgets.FloatText(description='VALOR ME:', style=style)
        self.moeda_dropdown = widgets.Dropdown(options=self.app_ref.CURRENCY_OPTIONS, description='MOEDA:', style=style)
        self.taxa_concorrencia_input = widgets.FloatText(description='TAXA CONCORRÊNCIA R$:', style=style)

        self.compra_button = widgets.Button(description="COMPRA", button_style='success', icon='arrow-down')
        self.venda_button = widgets.Button(description="VENDA", button_style='danger', icon='arrow-up')
        self.limpar_button = widgets.Button(description="LIMPAR", button_style='warning', icon='trash')

        self.output_display = widgets.HTML()

        self.compra_button.on_click(self._calcular)
        self.venda_button.on_click(self._calcular)
        self.limpar_button.on_click(self.limpar)

        botoes_box = widgets.HBox([self.compra_button, self.venda_button, self.limpar_button])

        self.layout.children = [
            info_html,
            self.tarifa_custo_input,
            self.valor_me_input,
            widgets.HTML("<hr>"),
            self.moeda_dropdown,
            self.taxa_concorrencia_input,
            botoes_box,
            widgets.HTML("<hr>"),
            self.output_display
        ]
        self.limpar(None)

    def _calcular(self, b):
        self.app_ref.increment_click_count()
        with debug_output:
            clear_output(wait=True)
            self.output_display.value = "<i>Buscando cotação...</i>"

            moeda = self.moeda_dropdown.value
            taxa_concorrencia = parse_input(self.taxa_concorrencia_input.value)
            tarifa_custo = parse_input(self.tarifa_custo_input.value)
            valor_me = parse_input(self.valor_me_input.value)

            is_taxa_mode = taxa_concorrencia > 0
            is_custo_mode = tarifa_custo > 0 and valor_me > 0

            if not is_taxa_mode and not is_custo_mode:
                self.output_display.value = "<b style='color:red;'>Erro: Preencha a 'TAXA CONCORRÊNCIA' ou os campos 'TARIFA/CUSTO' e 'VALOR ME'.</b>"
                return

            try:
                taxa_cambial, _ = self.app_ref._get_latest_bcb_rate(moeda)
                if taxa_cambial == 0:
                    self.output_display.value = f"<b style='color:red;'>Não foi possível obter a taxa PTAX para {moeda}.</b>"
                    return

                spread_pct = 0

                if is_taxa_mode:
                    if b.description == 'COMPRA':
                        spread_pct = ((taxa_cambial - taxa_concorrencia) / taxa_cambial) * 100
                    else: # VENDA
                        spread_pct = ((taxa_concorrencia - taxa_cambial) / taxa_cambial) * 100
                elif is_custo_mode:
                    if valor_me == 0:
                        self.output_display.value = "<b style='color:red;'>Erro: VALOR ME não pode ser zero.</b>"; return
                    spread_pct = ((tarifa_custo / valor_me) / taxa_cambial) * 100

                if spread_pct < 0:
                    cor_spread = 'red'
                    mensagem_prejuizo = "<br><b style='color:red;'>OPERAÇÃO COM PREJUÍZO</b>"
                else:
                    cor_spread = 'blue'
                    mensagem_prejuizo = ""

                self.output_display.value = f"""
                <div style='background-color:#f0f7ff; padding:10px; border-radius:5px; line-height: 1.6;'>
                    <h4>Resultado do Spread</h4>
                    <b>Moeda:</b> {moeda}<br>
                    <b>TAXA CAMBIAL (SPOT) R$:</b> {formatar_generico(taxa_cambial, 4)}<br>
                    <hr style='margin: 5px 0;'>
                    <b style='font-size: 1.2em;'>SPREAD DA CONCORRÊNCIA: <span style='color:{cor_spread};'>{formatar_generico(spread_pct, 2)}%</span></b>
                    {mensagem_prejuizo}
                </div>
                """
            except Exception as e:
                self.output_display.value = f"<b style='color:red;'>Erro ao calcular. Verifique o log.</b>"
                print(f"Erro em ModuloSpreadConcorrencia._calcular:\n{traceback.format_exc()}")

    def limpar(self, b):
        self.tarifa_custo_input.value = 0
        self.valor_me_input.value = 0
        self.taxa_concorrencia_input.value = 0
        self.output_display.value = ""

class ModuloPTAX(ModuloBase):
    """Módulo para consultar cotações históricas da PTAX."""
    CURRENCY_OPTIONS_PTAX = [
        ('Dólar Americano', 'USD'), ('Dólar Australiano', 'AUD'), ('Dólar Canadense', 'CAD'),
        ('Coroa Dinamarquesa', 'DKK'), ('Euro', 'EUR'), ('Iene', 'JPY'),
        ('Libra Esterlina', 'GBP'), ('Coroa Norueguesa', 'NOK'), ('Franco Suíço', 'CHF'),
        ('Coroa Sueca', 'SEK')
    ]
    def __init__(self, app_ref):
        super().__init__()
        self.app_ref = app_ref
        style = {'description_width': 'initial'}

        today = date.today()
        dias_uteis_para_voltar = 2
        data_inicial_padrao = today
        dias_contados = 0
        while dias_contados < dias_uteis_para_voltar:
            data_inicial_padrao -= timedelta(days=1)
            if data_inicial_padrao.weekday() < 5: # 0-4 são Seg-Sex
                dias_contados += 1

        info_html = widgets.HTML("<h4>Consulta de Cotações (PTAX)</h4><i>Selecione o período e a moeda para consultar o histórico oficial do Banco Central.</i>")
        self.data_inicial_picker = widgets.DatePicker(description='Data Inicial', value=data_inicial_padrao, style=style)
        self.data_final_picker = widgets.DatePicker(description='Data Final', value=date.today(), style=style)
        self.moeda_dropdown = widgets.Dropdown(options=self.CURRENCY_OPTIONS_PTAX, description='Moeda:', style=style)

        self.consultar_button = widgets.Button(description="CONSULTAR", button_style='primary', icon='search')
        self.limpar_button = widgets.Button(description="LIMPAR", button_style='danger', icon='trash')

        self.output_area = widgets.HTML()

        self.consultar_button.on_click(self._consultar)
        self.limpar_button.on_click(self._limpar)

        botoes_box = widgets.HBox([self.consultar_button, self.limpar_button])

        self.layout.children = [
            info_html,
            self.data_inicial_picker,
            self.data_final_picker,
            self.moeda_dropdown,
            botoes_box,
            widgets.HTML("<hr>"),
            self.output_area
        ]

    def _limpar(self, b):
        self.data_inicial_picker.value = None
        self.data_final_picker.value = date.today()
        self.output_area.value = ""

    def _consultar(self, b):
        self.app_ref.increment_click_count()
        with debug_output:
            clear_output(wait=True)
            data_ini = self.data_inicial_picker.value
            data_fim = self.data_final_picker.value
            moeda_simbolo = self.moeda_dropdown.value
            if not data_ini or not data_fim:
                self.output_area.value = "<b style='color:red;'>Erro: As datas inicial e final devem ser preenchidas.</b>"
                return
            if data_ini > data_fim:
                self.output_area.value = "<b style='color:red;'>Erro: A data inicial não pode ser posterior à data final.</b>"
                return
            data_ini_str = data_ini.strftime('%m-%d-%Y')
            data_fim_str = data_fim.strftime('%m-%d-%Y')
            self.output_area.value = "<i>Consultando o Serviço do Banco Central...</i>"

            url = (
                f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
                f"CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?"
                f"@moeda='{moeda_simbolo}'&@dataInicial='{data_ini_str}'&@dataFinalCotacao='{data_fim_str}'&$format=json"
                f"&$filter=tipoBoletim eq 'Fechamento'"
            )

            try:
                response = requests.get(url, timeout=20)
                response.raise_for_status()
                data = response.json()
                cotacoes = data.get('value', [])
                if not cotacoes:
                    self.output_area.value = "<p>Nenhuma cotação encontrada para o período e moeda selecionados.</p>"
                    return
                html_table = """
                <style>
                    .ptax-table { width: 100%; border-collapse: collapse; font-family: Arial, sans-serif; }
                    .ptax-table th, .ptax-table td { border: 1px solid #ddd; padding: 8px; text-align: center; }
                    .ptax-table th { background-color: #f2f2f2; }
                    .ptax-table tr:nth-child(even) { background-color: #f9f9f9; }
                </style>
                <table class='ptax-table'>
                    <thead>
                        <tr>
                            <th>Data</th>
                            <th>Cotação de Compra</th>
                            <th>Cotação de Venda</th>
                        </tr>
                    </thead>
                    <tbody>
                """
                for item in cotacoes:
                    data_hora = datetime.fromisoformat(item['dataHoraCotacao']).strftime('%d/%m/%Y')
                    html_table += (
                        f"<tr>"
                        f"<td>{data_hora}</td>"
                        f"<td>{formatar_generico(item['cotacaoCompra'], 4)}</td>"
                        f"<td>{formatar_generico(item['cotacaoVenda'], 4)}</td>"
                        f"</tr>"
                    )
                html_table += "</tbody></table>"
                footer_text = """
                <div style='font-size: 0.8em; color: #555; margin-top: 15px;'>
                    <p style='margin: 2px 0;'>1/&nbsp;- Moeda contra Real</p>
                    <p style='margin: 2px 0;'>4/&nbsp;- Fechamento Ptax = A partir de 1/7/2011, é a média aritmética das taxas de compra e das taxas de venda dos boletins do dia, conforme Circulares 3506, de 23/9/10, e 3537, de 25/5/11. Até 30/6/2011, é a taxa média ponderada dos negócios realizados no mercado interbancário de câmbio com liquidação em dois dias úteis, calculada pelo Banco Central do Brasil, conforme Comunicado N. 6815/99.</p>
                </div>
                """
                self.output_area.value = html_table + footer_text

            except requests.exceptions.RequestException as e:
                self.output_area.value = f"<b style='color:red'>Erro de conexão: Não foi possível consultar o serviço do Banco Central.</b>"
                print(f"Erro de conexão com a API PTAX: {e}")
            except Exception as e:
                self.output_area.value = f"<b style='color:red;'>Ocorreu um erro inesperado ao processar a consulta.</b>"
                print(f"Erro inesperado no Módulo PTAX: \n{traceback.format_exc()}")

class ModuloConversor(ModuloBase):
    """Módulo para converter valores entre diferentes moedas usando a PTAX como base."""
    BCB_CURRENCY_LIST = [
        ('Real Brasileiro', 'BRL'),
        ('Dólar Americano', 'USD'),
        ('Euro', 'EUR'),
        ('Libra Esterlina', 'GBP'),
        ('Iene Japonês', 'JPY'),
        ('Dólar Canadense', 'CAD'),
        ('Dólar Australiano', 'AUD'),
        ('Franco Suíço', 'CHF'),
        ('Coroa Dinamarquesa', 'DKK'),
        ('Coroa Norueguesa', 'NOK'),
        ('Coroa Sueca', 'SEK')
    ]

    def __init__(self, app_ref):
        super().__init__()
        self.app_ref = app_ref
        style = {'description_width': 'initial'}

        info_html = widgets.HTML("<h4>PARIDADE (Conversor de moedas)</h4><i>Use as taxas PTAX do BCB para converter valores entre diferentes moedas.</i>")
        self.valor_a_converter = widgets.FloatText(description='Valor:', value=1.0, style=style)
        self.moeda_de = widgets.Dropdown(options=self.BCB_CURRENCY_LIST, value='BRL', description='De:', style=style)
        self.moeda_para = widgets.Dropdown(options=self.BCB_CURRENCY_LIST, value='USD', description='Para:', style=style)

        self.calcular_button = widgets.Button(description="CALCULAR", button_style='success', icon='calculator')
        self.alternar_button = widgets.Button(description="ALTERNAR", button_style='info', icon='exchange')
        self.limpar_button = widgets.Button(description="LIMPAR", button_style='warning', icon='trash')

        self.output_display = widgets.HTML()

        self.calcular_button.on_click(self._calcular_paridade)
        self.alternar_button.on_click(self._alternar_moedas)
        self.limpar_button.on_click(self._limpar_campos)

        botoes_box = widgets.HBox([self.calcular_button, self.alternar_button, self.limpar_button])

        self.layout.children = [
            info_html,
            self.valor_a_converter,
            self.moeda_de,
            self.moeda_para,
            botoes_box,
            widgets.HTML("<hr>"),
            self.output_display
        ]

    def _alternar_moedas(self, b):
        de_val = self.moeda_de.value
        para_val = self.moeda_para.value
        self.moeda_de.value = para_val
        self.moeda_para.value = de_val

    def _limpar_campos(self, b):
        self.valor_a_converter.value = 1.0
        self.moeda_de.value = 'BRL'
        self.moeda_para.value = 'USD'
        self.output_display.value = ""

    def _get_label_from_code(self, code):
        return next((label for label, value in self.BCB_CURRENCY_LIST if value == code), code)

    def _calcular_paridade(self, b):
        self.app_ref.increment_click_count()

        valor = self.valor_a_converter.value
        moeda_de_code = self.moeda_de.value
        moeda_para_code = self.moeda_para.value

        if not valor or valor < 0:
            self.output_display.value = "<b style='color:red;'>Por favor, insira um valor válido.</b>"
            return

        self.output_display.value = "<i>Buscando cotações...</i>"

        try:
            taxa_de, data_cotacao_de = self.app_ref._get_latest_bcb_rate(moeda_de_code)
            taxa_para, data_cotacao_para = self.app_ref._get_latest_bcb_rate(moeda_para_code)

            if taxa_de == 0:
                self.output_display.value = f"<b style='color:red;'>Não foi possível obter a cotação para {moeda_de_code}.</b>"
                return
            if taxa_para == 0:
                self.output_display.value = f"<b style='color:red;'>Não foi possível obter a cotação para {moeda_para_code}.</b>"
                return

            taxa_paridade = taxa_de / taxa_para
            valor_convertido = valor * taxa_paridade

            valor_convertido_arredondado = round(valor_convertido, 2)

            data_cotacao = date.today().strftime('%d/%m/%Y')
            if data_cotacao_de and data_cotacao_para:
                data_cotacao = min(data_cotacao_de, data_cotacao_para).strftime('%d/%m/%Y')

            moeda_de_label = self._get_label_from_code(moeda_de_code)
            moeda_para_label = self._get_label_from_code(moeda_para_code)

            timestamp_paridade = datetime.now().strftime('%d/%m/%Y %H:%M:%S')

            html_output = f"""
            <div style='line-height: 1.8;'>
                <b>Resultado da conversão</b><br>
                Conversão de: <span style='color:blue; font-weight:bold;'>{moeda_de_label} / {moeda_de_code}</span> para <span style='color:green; font-weight:bold;'>{moeda_para_label} / {moeda_para_code}</span><br>
                Valor a converter: <span style='color:blue; font-weight:bold;'>{moeda_de_code} {formatar_me(valor)}</span><br>
                Para: <span style='color:green; font-weight:bold;'>{moeda_para_label}/{moeda_para_code}</span><br>
                Resultado da conversão: <span style='color:green; font-weight:bold;'>{moeda_para_code} {formatar_generico(valor_convertido_arredondado, 2)}</span><br>
                Data cotação utilizada: <b>{data_cotacao}</b><br>
                <hr style='margin: 10px 0;'>
                <b>Taxa:</b><br>
                1 {moeda_de_code} = <span style='color:green; font-weight:bold;'>{moeda_para_code} {formatar_generico(taxa_paridade, 7)}</span><br>
                1 {moeda_para_code} = <span style='color:blue; font-weight:bold;'>{moeda_de_code} {formatar_generico(1/taxa_paridade, 7)}</span><br>
                <hr style='margin: 10px 0;'>
                <b>PARIDADE</b><br>
                <span style='color:blue; font-weight:bold;'>{moeda_de_code} {formatar_me(valor)}</span> = <span style='color:green; font-weight:bold;'>{moeda_para_code} {formatar_generico(valor_convertido_arredondado, 2)}</span><br>
                <br>
                <i style='font-size: 0.8em;'>Cálculo de paridade realizado em: {timestamp_paridade}</i>
            </div>
            """
            self.output_display.value = html_output

        except Exception as e:
            self.output_display.value = "<b style='color:red;'>Ocorreu um erro ao calcular a paridade.</b>"
            with debug_output:
                print(f"Erro no Módulo Conversor: {traceback.format_exc()}")


# ==============================================================================
# 4. APLICAÇÃO PRINCIPAL
# ==============================================================================
class CalculadoraCambioApp:
    CURRENCY_OPTIONS = [('USD - Dolar Americano', 'USD'), ('EUR - Euro', 'EUR'), ('GBP - Libra Esterlina', 'GBP'), ('CHF - Franco Suiço', 'CHF'), ('CAD - Dólar Canadense', 'CAD'), ('AUD - Dólar Australiano', 'AUD'), ('JPY - Iene Japonês', 'JPY')]
    def __init__(self):
        self.op_selecionada = None; style={'description_width':'initial'}; self.app_is_running=True

        self.kpi_click_count = 0
        self.kpi_total_revenue = 0.0

        self.is_history_mode = False
        self.calculation_history = deque(maxlen=10)
        self.ticker_bar=widgets.HTML(value="")
        self.style_output = widgets.HTML(value="")

        self.moeda_selecionada=widgets.Dropdown(options=self.CURRENCY_OPTIONS, value='USD', description='Moeda:', style=style)
        self.valor_me=widgets.Text(description='Valor M.E.:',placeholder='Ex: 10000.00',style=style)
        self.spread=widgets.FloatText(description='Spread %:',value=0.9,step=0.1,style=style)
        button_layout = widgets.Layout(width='50%')
        self.compra_button=widgets.Button(description="COMPRA",button_style='success',icon='arrow-down', layout=button_layout)
        self.venda_button=widgets.Button(description="VENDA",button_style='danger',icon='arrow-up', layout=button_layout)

        # --- Componentes de Layout para o Gerenciador ---
        self.taxa_display_main = widgets.HTML(value="<i>Preencha os campos acima e selecione COMPRA ou VENDA.</i>")
        self.taxa_display_ptax = widgets.HTML()
        self.ptax_info_button = widgets.Button(
            icon='info-circle',
            tooltip='Por que esta taxa foi usada?',
            layout=widgets.Layout(visibility='hidden', width='35px', border='none', background='transparent')
        )
        self.ptax_info_output = widgets.HTML(value="", layout={'display': 'none'})

        self.gerenciador_layout=widgets.VBox([
            widgets.HBox([self.moeda_selecionada,self.valor_me]),
            self.spread,
            widgets.HBox([self.compra_button,self.venda_button]),
            self.taxa_display_main,
            widgets.HBox([self.taxa_display_ptax, self.ptax_info_button]),
            self.ptax_info_output
        ])

        self.mod_compra=ModuloCompra(self)
        self.mod_venda=ModuloVenda(self)
        self.mod_valor_em_reais=ModuloValorEmReais(self)
        self.mod_concomitante=ModuloConcomitante(self)
        self.mod_spread_concorrencia=ModuloSpreadConcorrencia(self)
        self.mod_ptax = ModuloPTAX(self)
        self.mod_conversor = ModuloConversor(self)

        self.chart_output = widgets.Output()
        self.moeda_selecionada.observe(self._update_chart, names='value')

        self.accordion_esquerda=widgets.Accordion(children=[self.mod_compra.get_layout(), self.mod_valor_em_reais.get_layout(), self.mod_conversor.get_layout()])
        self.accordion_esquerda.set_title(0,'COMPRA (RECEBIMENTO)')
        self.accordion_esquerda.set_title(1,'VALOR EM REAIS')
        self.accordion_esquerda.set_title(2,'PARIDADE (Conversor de moedas)')
        self.accordion_esquerda.selected_index=None

        self.accordion_direita=widgets.Accordion(children=[self.mod_venda.get_layout(),self.mod_concomitante.get_layout(),self.mod_spread_concorrencia.get_layout(), self.mod_ptax.get_layout()])
        self.accordion_direita.set_title(0,'VENDA (ENVIO)')
        self.accordion_direita.set_title(1,'CONCOMITANTE')
        self.accordion_direita.set_title(2,'SPREAD CONCORRÊNCIA')
        self.accordion_direita.set_title(3, '📈 PTAX (Consulta)')
        self.accordion_direita.selected_index=None

        coluna_esquerda_box=widgets.VBox([self.accordion_esquerda],layout=widgets.Layout(width='50%',border='1px solid #e0e0e0',padding='5px'))
        coluna_direita_box=widgets.VBox([self.accordion_direita],layout=widgets.Layout(width='50%',border='1px solid #e0e0e0',padding='5px'))
        self.modulos_layout=widgets.HBox([coluna_esquerda_box,coluna_direita_box])

        self.info_gabrielino_button = widgets.Button(
            icon='info-circle', tooltip='Sobre esta calculadora',
            layout=widgets.Layout(width='auto', border='none', background='transparent'))
        self.info_gabrielino_output = widgets.Output()
        self.info_gabrielino_visible = False
        self.info_gabrielino_button.on_click(self._toggle_info_gabrielino)

        self.dark_mode_button = widgets.ToggleButton(value=False, tooltip='Alternar Modo Escuro', icon='moon-o', button_style='')
        self.dark_mode_button.observe(self._toggle_dark_mode, names='value')

        self.kpi_output = widgets.Output()
        self.kpi_visible = False

        self.compra_button.on_click(self._on_compra_click)
        self.venda_button.on_click(self._on_venda_click)
        self.ptax_info_button.on_click(self._toggle_ptax_info)

        self.update_thread=threading.Thread(target=self._run_update_loop,daemon=True)
        self.update_thread.start()

        self._update_chart()

    def _toggle_ptax_info(self, b):
        """Mostra ou esconde a explicação sobre a regra da PTAX D-2."""
        if self.ptax_info_output.layout.display == 'none':
            info_message = """
            <div style='border: 1px solid #007bff; padding: 10px; border-radius: 5px; background-color: #f0f7ff; margin-top: 5px; color: black;'>
                <b>Base de Cálculo do Imposto de Renda (IRRF)</b>
                <p>Para o cálculo do imposto, utilizamos uma taxa de câmbio de referência baseada na PTAX (taxa divulgada pelo Banco Central).</p>
                <p>Conforme a legislação (Lei Nº 9.816, Art. 3º), a taxa de câmbio para tributação do IRRF é o <b>MAIOR</b> valor entre:</p>
                <ul style='margin-left: 20px; padding-left: 0;'>
                    <li>A taxa de câmbio comercial com spread bancário oferecida pela instituição financeira.</li>
                    <li>A taxa PTAX de venda referente a 2 (dois) dias úteis anteriores à data de criação da sua operação.</li>
                </ul>
                <p style='margin-top: 5px;'>Neste caso, a PTAX de D-2 foi maior e, portanto, utilizada como base para o cálculo do imposto.</p>
            </div>
            """
            self.ptax_info_output.value = info_message
            self.ptax_info_output.layout.display = 'block'
        else:
            self.ptax_info_output.layout.display = 'none'

    def _toggle_dark_mode(self, change):
        if change['new']:
            self.style_output.value = self._get_dark_mode_css()
            self.dark_mode_button.icon = 'sun-o'
        else:
            self.style_output.value = ""
            self.dark_mode_button.icon = 'moon-o'

    def increment_click_count(self):
        self.kpi_click_count += 1

    def add_revenue(self, revenue):
        if revenue > 0:
            self.kpi_total_revenue += revenue

    def _toggle_kpi_display(self, b):
        self.kpi_visible = not self.kpi_visible
        with self.kpi_output:
            clear_output()
            if self.kpi_visible:
                kpi_text = f"""
                <div style='border: 1px solid #FFC107; padding: 10px; border-radius: 5px; background-color: #FFF9C4;'>
                    <p style=\"margin:0; line-height:1.4; color: black; font-weight: bold;\">
                        Esta calculadora ja foi utilizada <span style='background-color: yellow; padding: 2px 4px; border-radius: 3px;'>{self.kpi_click_count}</span> vezes.
                    </p>
                    <p style=\"margin:5px 0 0 0; line-height:1.4; color: black; font-weight: bold;\">
                        Auxiliamos o banco em receita de <span style='background-color: yellow; padding: 2px 4px; border-radius: 3px;'>R$ {formatar_brl(self.kpi_total_revenue)}</span>
                    </p>
                </div>
                """
                display(HTML(kpi_text))

    def _toggle_info_gabrielino(self, b):
        self.info_gabrielino_visible = not self.info_gabrielino_visible
        with self.info_gabrielino_output:
            clear_output()
            if self.info_gabrielino_visible:
                info_text = """
                <div style='border: 1px solid #ccc; padding: 10px; border-radius: 5px; background-color: #f9f9f9; margin-top: 5px;'>
                    <p style=\"margin:0; line-height:1.4;\"><b>Calculadora criada por Gabriel Roseno</b><br>
                    RACF: ROSENOG - Funcional 987294026</p>
                    <hr style='margin: 8px 0;'>
                    <p style=\"margin:0; line-height:1.4;\">Tem alguma reclamação ou sugestão? Me mande um e-mail que será muito bem vindo, vamos juntos melhorar essa ferramenta.</p>
                    <p style=\"margin:8px 0 0 0;\"><i>#vamosdeturma</i></p>
                </div>
                """
                display(HTML(info_text))

    def _get_dark_mode_css(self):
        return """<style>
        .app-container { background-color: #2d2d2d !important; color: #f0f0f0 !important; }
        .app-container .p-Accordion-content { background-color: #f9f9f9 !important; color: #111 !important; border-color: #555 !important; }
        .app-container .widget-label, .app-container .widget-html-content, .app-container h4, .app-container .p-Accordion-title { color: #f0f0f0 !important; }
        .app-container .p-Accordion-content .widget-label, .app-container .p-Accordion-content .widget-html-content, .app-container .p-Accordion-content h4, .app-container .p-Accordion-content b, .app-container .p-Accordion-content i { color: #111 !important; }
        .app-container .p-Accordion-content span[style*='color: blue'] { color: blue !important; }
        .app-container .p-Accordion-content span[style*='color: green'] { color: green !important; }
        .app-container .p-Accordion-content span[style*='color: purple'] { color: purple !important; }
        .app-container .p-Accordion-content span[style*='color:red'] { color: red !important; }
        .app-container .p-Accordion-content span[style*='color:black'] { color: black !important; }

        .app-container .widget-text input, .app-container .widget-float-text input, .app-container .widget-dropdown select { background-color: #444 !important; color: #f0f0f0 !important; border: 1px solid #555 !important; }
        .app-container .widget-button { background-color: #5a5a5a !important; border-color: #6a6a6a !important; color: #f0f0f0 !important; }
        .app-container .widget-button:hover { background-color: #6a6a6a !important; }
        .app-container .p-Accordion-title { background-color: #383838 !important; color: orange !important; }
        .app-container .dark-mode-text-override, .app-container .dark-mode-text-override span, .app-container .dark-mode-text-override b { color: black !important; }
        .app-container .dark-mode-text-override span[style*='color: blue'] { color: blue !important; }
        .app-container .dark-mode-text-override span[style*='color: purple'] { color: purple !important; }
        .app-container .dark-mode-text-override span[style*='color: green'] { color: green !important; }
        .app-container .dark-mode-text-override b[style*='color: #c00'] { color: #c00 !important; }
        </style>"""

    def _get_app_css(self):
        return """<style>
            .itau-font-app, .itau-font-app .jupyter-widget { font-family: 'Itau display', 'Itau', Arial, sans-serif !important; }
            @keyframes ticker-scroll { from { transform: translateX(0); } to { transform: translateX(-50%); } }
            .ticker-wrap { position: relative; width: 100%; overflow: hidden; background-color: #111; color: #fff; box-sizing: border-box; padding: 10px 0; border-bottom: 3px solid #111; }
            .ticker-content { display: flex; width: fit-content; animation: ticker-scroll 130s linear infinite; }
            .ticker-content:hover { animation-play-state: paused; }
            .ticker-item { flex-shrink: 0; margin: 0 25px; font-family: 'Courier New', Courier, monospace; font-size: 18px; white-space: nowrap; }
            .ticker-item b { color: #FFA500; }
            .ticker-item .last { color: #fff; font-weight: bold; }
            .ticker-item .time { color: #888; font-size: 11px; margin-left: 5px; }
        </style>"""

    def _update_history_ticker(self):
        """Atualiza a barra superior para mostrar o histórico de pesquisas."""
        if not self.calculation_history:
            self._update_market_ticker()
            return

        ticker_items_html = ""
        for item in reversed(self.calculation_history):
            ticker_items_html += (
                f"<div class='ticker-item'>"
                f"<b>{item['moeda']}/BRL</b> "
                f"<span class='last'>R$ {formatar_generico(item['taxa_pura'], 4)}</span> "
                f"<span class='time'>({item['timestamp']})</span>"
                f"</div>"
            )
        self.ticker_bar.value = f"<div class='ticker-wrap'><div class='ticker-content'>{ticker_items_html}{ticker_items_html}</div></div>"

    def _get_bcb_rate_for_date(self, currency_code, target_date):
        """Busca a cotação PTAX para uma moeda em uma data específica."""
        data_str = target_date.strftime('%m-%d-%Y')
        if currency_code == 'USD':
            url = f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarDia(dataCotacao=@dataCotacao)?@dataCotacao='{data_str}'&$format=json"
        else:
            url = f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaDia(moeda='{currency_code}',dataCotacao=@dataCotacao)?@dataCotacao='{data_str}'&$format=json"

        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200:
                data = response.json().get('value', [])
                if data:
                    compra = data[0].get('cotacaoCompra', 0) or 0
                    venda = data[0].get('cotacaoVenda', 0) or 0
                    return {'compra': compra, 'venda': venda}
        except requests.exceptions.RequestException:
            pass
        return None

    def _get_latest_bcb_rate(self, currency_code):
        """Busca a última cotação de fechamento PTAX para uma moeda, retornando a taxa e a data."""
        if currency_code == 'BRL':
            return 1.0, date.today()

        for i in range(7):
            query_date = date.today() - timedelta(days=i)
            rates = self._get_bcb_rate_for_date(currency_code, query_date)
            if rates and rates.get('compra'):
                return rates['compra'], query_date
        return 0, None

    def _update_market_ticker(self):
        """Atualiza a barra superior com dados de mercado PTAX do BCB."""
        try:
            ticker_items_html = ""
            for label, code in self.CURRENCY_OPTIONS:
                taxa, _ = self._get_latest_bcb_rate(code)
                if taxa > 0:
                    ticker_items_html += f"<div class='ticker-item'><b>{code}/BRL</b> <span class='last'>R$ {formatar_generico(taxa, 4)}</span></div>"
                else:
                    ticker_items_html += f"<div class='ticker-item'><b>{code}/BRL</b> <span class='last' style='color:red;'>N/A</span></div>"
            self.ticker_bar.value = f"<div class='ticker-wrap'><div class='ticker-content'>{ticker_items_html}{ticker_items_html}</div></div>"
        except Exception:
            with debug_output:
                print(f"Erro ao atualizar a barra de cotações de mercado:\n{traceback.format_exc()}")
            self.ticker_bar.value = "<div class='ticker-wrap'><div class='ticker-content' style='animation: none;'><div class='ticker-item' style='color:red;'>Erro ao carregar cotações do BCB.</div></div></div>"

    def _run_update_loop(self):
        """Loop que controla a atualização da barra superior."""
        if not self.is_history_mode:
            self._update_market_ticker()

        while self.app_is_running:
            time.sleep(300)
            if not self.is_history_mode:
                try:
                    self._update_market_ticker()
                except Exception as e:
                    with debug_output:
                        print(f"Erro no loop de atualização da barra: {e}")

    def _update_chart(self, change=None):
        """Busca dados da PTAX e desenha o gráfico histórico."""
        with self.chart_output:
            clear_output(wait=True)
            currency_code = self.moeda_selecionada.value
            print(f"Gerando gráfico para {currency_code}...")

            end_date = date.today()
            start_date = end_date - timedelta(days=15)

            data_ini_str = start_date.strftime('%m-%d-%Y')
            data_fim_str = end_date.strftime('%m-%d-%Y')

            url = (
                f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
                f"CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?"
                f"@moeda='{currency_code}'&@dataInicial='{data_ini_str}'&@dataFinalCotacao='{data_fim_str}'&$format=json"
                f"&$filter=tipoBoletim eq 'Fechamento'"
            )

            try:
                response = requests.get(url, timeout=15)
                response.raise_for_status()
                data = response.json().get('value', [])

                if not data:
                    print(f"Não foram encontrados dados de PTAX para {currency_code} nos últimos 15 dias.")
                    return

                dates = [datetime.fromisoformat(item['dataHoraCotacao']) for item in data]
                rates = [item['cotacaoVenda'] for item in data]

                fig, ax = plt.subplots(figsize=(10, 4), dpi=100)
                ax.plot(dates, rates, marker='o', linestyle='-', color='royalblue')

                ax.set_title(f'Histórico PTAX (Venda) - {currency_code}', fontsize=14)
                ax.set_ylabel('Cotação (R$)')
                ax.grid(True, linestyle='--', alpha=0.6)

                ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m'))
                ax.xaxis.set_major_locator(mdates.AutoDateLocator())
                fig.autofmt_xdate()

                fig.tight_layout()
                display(fig)
                plt.close(fig)

            except Exception as e:
                print(f"Falha ao gerar gráfico: {e}")

    def _get_venda_ir_base_rate(self, currency_code, effective_rate):
        """
        Verifica a PTAX de D-2 para operações de Venda.
        Retorna um dicionário com a maior taxa e a data se a PTAX for maior que a taxa da operação.
        Caso contrário, retorna None.
        """
        datas_uteis = []
        data_atual = date.today()
        while len(datas_uteis) < 2:
            data_atual -= timedelta(days=1)
            if data_atual.weekday() < 5:
                datas_uteis.append(data_atual)

        ptax_d1_data = self._get_bcb_rate_for_date(currency_code, datas_uteis[0])
        ptax_d2_data = self._get_bcb_rate_for_date(currency_code, datas_uteis[1])

        ptax_d1_taxa = ptax_d1_data.get('venda', 0) if ptax_d1_data else 0
        ptax_d2_taxa = ptax_d2_data.get('venda', 0) if ptax_d2_data else 0

        maior_taxa_historica = max(ptax_d1_taxa, ptax_d2_taxa)

        if maior_taxa_historica > effective_rate:
            data_maior_taxa = datas_uteis[0] if maior_taxa_historica == ptax_d1_taxa else datas_uteis[1]
            return {
                "rate": maior_taxa_historica,
                "date": data_maior_taxa
            }
        return None

    def _update_button_styles(self): self.compra_button.button_style='info' if self.op_selecionada=='COMPRA' else 'success'; self.venda_button.button_style='info' if self.op_selecionada=='VENDA' else 'danger'
    def _on_compra_click(self,b): self.op_selecionada='COMPRA'; self._update_button_styles(); self._executar_calculo_e_preparar()
    def _on_venda_click(self,b): self.op_selecionada='VENDA'; self._update_button_styles(); self._executar_calculo_e_preparar()

    def _executar_calculo_e_preparar(self):
        self.increment_click_count()
        with debug_output:
            # Reseta o estado da interface a cada cálculo
            clear_output(wait=True)
            self.ptax_info_button.layout.visibility = 'hidden'
            self.ptax_info_output.layout.display = 'none'
            self.taxa_display_ptax.value = ""
            self.compra_button.disabled=True
            self.venda_button.disabled=True
            try:
                valor_me = parse_input(self.valor_me.value)
                if valor_me <= 0:
                    self.taxa_display_main.value = "<b style='color:red'>Para continuar, insira o valor M.E (Moeda Estrangeira).</b>"
                    self.compra_button.disabled=False; self.venda_button.disabled=False; return

                moeda = self.moeda_selecionada.value
                self.taxa_display_main.value = f"<i>Buscando cotação PTAX para {moeda}...</i>"

                taxa_pura, _ = self._get_latest_bcb_rate(moeda)

                if not taxa_pura or taxa_pura == 0:
                    self.taxa_display_main.value = f"<b style='color:red'>Não foi possível obter a taxa PTAX para {moeda}.</b>"
                    self.compra_button.disabled=False; self.venda_button.disabled=False; return

                spread = self.spread.value
                is_compra = self.op_selecionada == 'COMPRA'
                taxa_efetiva = taxa_pura * (1-(spread/100) if is_compra else 1+(spread/100))

                if is_compra:
                    iof_widget = self.mod_compra.iof_compra
                    op_char_spread = "-"
                    op_char_iof = "-"
                    vet_calculado = taxa_efetiva * (1 - iof_widget.value)
                else:
                    iof_widget = self.mod_venda.iof_venda
                    op_char_spread = "+"
                    op_char_iof = "+"
                    vet_calculado = taxa_efetiva * (1 + iof_widget.value)

                iof_label = next((label for label, value in iof_widget.options if value == iof_widget.value), "")
                target = self.mod_compra if is_compra else self.mod_venda

                if is_compra:
                    self.accordion_esquerda.selected_index=0
                    self.accordion_direita.selected_index = None
                else:
                    self.accordion_esquerda.selected_index=None
                    self.accordion_direita.selected_index=0

                target.limpar_state()
                target.valor_me_local.value=valor_me
                target.taxa_cambial_local.value=round(taxa_efetiva, 4)
                target.taxa_comercial_hidden.value=taxa_pura

                target.calcular(None)

                # --- Lógica de exibição no Gerenciador ---
                # Exibe os resultados principais
                main_display_html = f"""
                <div class='dark-mode-text-override' style='background-color:#f0f7ff; padding:10px; border-radius:5px; line-height: 1.6;'>
                    <span>Taxa Comercial (SPOT) R$ {formatar_generico(taxa_pura, 4)}</span><br>
                    <span style='font-size: 1.1em; font-weight: bold; color: blue;'>Taxa Cambial: R$ {formatar_generico(taxa_efetiva, 4)}</span>
                    <span style='font-size: 0.9em;'> ({op_char_spread}{spread:.2f}% spread)</span><br>
                    <span style='font-weight: bold; color: purple;'>V.E.T R$ {formatar_generico(vet_calculado, 4)}</span>
                    <span style='font-size: 0.9em;'> ({op_char_iof} {iof_label} IOF)</span>
                </div>"""
                self.taxa_display_main.value = main_display_html

                # Exibe o aviso da PTAX D-2 e o botão de info, se aplicável
                if not is_compra:
                    ptax_check_result = self._get_venda_ir_base_rate(moeda, taxa_efetiva)
                    if ptax_check_result:
                        self.ptax_info_button.layout.visibility = 'visible'
                        ptax_warning_html = f"<b style='color: #c00;'>MAIOR ENTRE PTAX (D-2) R$ {formatar_generico(ptax_check_result['rate'], 4)} - {ptax_check_result['date'].strftime('%d/%m/%Y')}</b>"
                        self.taxa_display_ptax.value = ptax_warning_html

                self.is_history_mode = True
                self.calculation_history.append({
                    'moeda': moeda,
                    'taxa_pura': taxa_pura,
                    'taxa_efetiva': taxa_efetiva,
                    'timestamp': datetime.now().strftime('%H:%M')
                })
                self._update_history_ticker()

            except Exception: self.taxa_display_main.value="<b style='color:red'>Erro. Verifique o Log.</b>"; print(traceback.format_exc())
            finally: self.compra_button.disabled=False; self.venda_button.disabled=False

    def start(self):
        # --- TÍTULO COMO TEXTO SIMPLES ---
        title_label = widgets.HTML("<b>Bem vindo a Calculadora de Câmbio do Gabrielino</b>")

        # HBox para alinhar o título e os botões de info e dark mode
        title_box = widgets.HBox([
            title_label,
            widgets.HTML("<div style='flex-grow: 1;'></div>"),
            self.info_gabrielino_button,
            self.dark_mode_button
        ])

        # --- BOTÃO SECRETO INVISÍVEL ---
        secret_kpi_button = widgets.Button(
            description=".",
            layout={'width': 'auto'},
            button_style='' # Remove o estilo de botão para parecer invisível
        )
        secret_kpi_button.style.button_color = 'transparent'
        secret_kpi_button.style.border_color = 'transparent'
        secret_kpi_button.on_click(self._toggle_kpi_display)

        # Box para alinhar o botão secreto à direita
        secret_button_box = widgets.HBox(
            [secret_kpi_button],
            layout=widgets.Layout(width='100%', justify_content='flex-end')
        )

        main_container = widgets.VBox([
            self.style_output,
            self.ticker_bar,
            title_box,
            self.info_gabrielino_output,
            self.kpi_output,
            self.gerenciador_layout,
            self.modulos_layout,
            widgets.HTML("<hr><h4>Gráfico Histórico (Últimos 15 dias)</h4>"),
            self.chart_output,
            secret_button_box,
            widgets.HTML("<hr><h4>Log de Erros</h4>"),
            debug_output
        ], layout=widgets.Layout(margin='0 auto', width='90%'))
        main_container.add_class('app-container')

        display(HTML(self._get_app_css()), main_container)
        self._update_chart()

if __name__ == "__main__":
    app=CalculadoraCambioApp()
    app.start()